In [1]:
import os
import pandas as pd
import numpy as np
import glob

In [2]:
amg_contigs_for_checking = [f.replace('__', '||') for f in np.loadtxt('/fs/ess/PAS0439/MING/virome/amg_analysis/amg_containing_contigs_curated_vs2.txt', dtype = 'str')]

In [55]:
amr_contigs_for_checking = np.loadtxt('/fs/ess/PAS0439/MING/virome/AMR_AMG_containing_contigs_manual_check/amr_containing_contigs.txt', dtype = 'str')

In [5]:
os.chdir('/fs/ess/PAS0439/MING/virome/checkv_res')

In [6]:
checkv_files = glob.glob('**/quality_summary.tsv', recursive = True)

In [7]:
checkv_df = []
for file in checkv_files:
    df = pd.read_csv(file, sep = '\t')
    checkv_df.append(df)

In [8]:
checkv_for_manual_curation = pd.concat(checkv_df)

In [9]:
checkv_for_amg_curation = checkv_for_manual_curation[checkv_for_manual_curation.contig_id.isin(amg_contigs_for_checking)][['contig_id', 'contig_length', 'gene_count', 'viral_genes', 'host_genes', 'checkv_quality']].reset_index(drop = True)

In [10]:
checkv_for_amg_curation

,contig_id,contig_length,gene_count,viral_genes,host_genes,checkv_quality
0,HS.02_67179||full,185095,315,32,4,Complete
1,HS.02_78242||full,190788,339,42,3,Complete
2,HS.02_441524||full,90992,151,15,5,Complete
3,HS.02_493423||full,68833,106,16,1,Complete
4,HS.02_781309||full,173063,277,35,5,Complete
...,...,...,...,...,...,...
1879,ERR4297101_1715298||full,63025,99,19,2,Complete
1880,ERR4297101_1718598||full,58144,95,21,1,Complete
1881,SRR12021134_56306||full,52438,76,21,3,Complete
1882,SRR12021134_2084708||full,118286,142,30,3,Complete


In [11]:
checkv_for_amr_curation = checkv_for_manual_curation[checkv_for_manual_curation.contig_id.isin(amr_contigs_for_checking)][['contig_id', 'contig_length', 'gene_count','viral_genes','host_genes', 'checkv_quality']].reset_index(drop = True)

In [12]:
#pd.set_option('display.max_rows', None)
checkv_for_amr_curation.query('~(host_genes > 0 & viral_genes == 0)').query('contig_length > 10000')

,contig_id,contig_length,gene_count,viral_genes,host_genes,checkv_quality
0,HS.02_1005742||full,24267,26,10,0,Low-quality
1,SRR4435581_160778||0_partial,38079,44,6,11,Low-quality
2,SRR4435563_250538||full,23553,30,6,1,Low-quality
3,SRR12529381_6541||full,43161,53,8,1,Low-quality
4,SRR12529381_382437||full,28430,32,4,11,Low-quality
...,...,...,...,...,...,...
322,ERR3515520_65914||full,24424,26,3,10,Low-quality
323,SRR8397906_198667||0_partial,13359,16,2,3,Low-quality
328,ERR4297094_720966||full,10226,13,1,2,Low-quality
329,ERR4297090_449111||full,20559,16,1,1,Low-quality


In [13]:
os.chdir('/fs/ess/PAS0439/MING/virome/amg_analysis/dram_annotation')

In [14]:
dramv_amg = glob.glob('**/annotations.tsv', recursive = True)

In [15]:
dramv_amg_df = []
for file in dramv_amg:
    df = pd.read_csv(file, sep = '\t')
    dramv_amg_df.append(df)

/tmp/slurmtmp.11695650/ipykernel_268136/1893469611.py:3: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, sep = '\t')


In [16]:
dramv_amg_for_manual_curation = pd.concat(dramv_amg_df)

In [17]:
dramv_amg_for_manual_curation.rename(columns = {'Unnamed: 0': 'gene_name'}, inplace = True)
dramv_amg_for_manual_curation.fasta = dramv_amg_for_manual_curation.fasta.apply(lambda x: x.replace('__full-cat_1', '||full'))

In [18]:
dramv_amg_for_manual_curation_reformat = dramv_amg_for_manual_curation.get(['gene_name', 'fasta', 'kegg_hit', 'viral_hit', 'pfam_hits'])
dramv_amg_for_manual_curation_reformat_new = dramv_amg_for_manual_curation_reformat[dramv_amg_for_manual_curation_reformat.fasta.isin(amg_contigs_for_checking)]

In [19]:
group = dramv_amg_for_manual_curation_reformat_new.groupby('fasta').groups.keys()

In [20]:
import re
def find_text(ser, string):
    res = ser[ser.str.contains(string, flags=re.IGNORECASE)]
    if res.empty:
        res = ''
    else:
        res = str(list(res))
    return res
def get_hallmark(df):
    kegg_hit = df['kegg_hit'].fillna('')
    viral_hit = df['viral_hit'].fillna('')
    pfam_hit =  df['pfam_hits'].fillna('')
    
       
    kegg_capsid = find_text(kegg_hit, 'capsid')
    kegg_portal = find_text(kegg_hit, 'portal')
    kegg_coat = find_text(kegg_hit, 'portal')
    kegg_baseplate = find_text(kegg_hit, 'baseplate')
    kegg_head = find_text(kegg_hit, 'head')
    kegg_tail = find_text(kegg_hit, 'tail')
    kegg_tail_fiber = find_text(kegg_hit, 'tail fiber')
    kegg_terminase = find_text(kegg_hit, 'terminase')
    kegg_integrase = find_text(kegg_hit, 'integrase')
    kegg_transposase = find_text(kegg_hit, 'transposase')
    
    viral_capsid = find_text(viral_hit, 'capsid')
    viral_portal = find_text(viral_hit, 'portal')
    viral_coat = find_text(viral_hit, 'portal')
    viral_baseplate = find_text(viral_hit, 'baseplate')
    viral_head = find_text(viral_hit, 'head')
    viral_tail = find_text(viral_hit, 'tail')
    viral_tail_fiber = find_text(viral_hit, 'tail fiber')
    viral_terminase = find_text(viral_hit, 'terminase')
    viral_integrase = find_text(viral_hit, 'integrase')
    viral_transposase = find_text(viral_hit, 'transposase')
    
    pfam_capsid = find_text(pfam_hit, 'capsid')
    pfam_portal = find_text(pfam_hit, 'portal')
    pfam_coat = find_text(pfam_hit, 'portal')
    pfam_baseplate = find_text(pfam_hit, 'baseplate')
    pfam_head = find_text(pfam_hit, 'head')
    pfam_tail = find_text(pfam_hit, 'tail')
    pfam_tail_fiber = find_text(pfam_hit, 'tail fiber')
    pfam_terminase = find_text(pfam_hit, 'terminase')
    pfam_integrase = find_text(pfam_hit, 'integrase')
    pfam_transposase = find_text(pfam_hit, 'transposase')
    return kegg_capsid, kegg_portal, kegg_coat, kegg_baseplate, kegg_head, kegg_tail, kegg_tail_fiber, kegg_terminase, kegg_integrase, kegg_transposase, viral_capsid, viral_portal, viral_coat, viral_baseplate, viral_head, viral_tail, viral_tail_fiber, viral_terminase, viral_integrase, viral_transposase, pfam_capsid, pfam_portal, pfam_coat, pfam_baseplate, pfam_head, pfam_tail, pfam_tail_fiber, pfam_terminase, pfam_integrase, pfam_transposase
        
    
    

In [21]:
dramv_amg_for_manual_curation_final = pd.DataFrame(list(dramv_amg_for_manual_curation_reformat_new.groupby('fasta').apply(get_hallmark)), index = group, columns = ['kegg_capsid', 'kegg_portal', 'kegg_coat', 'kegg_baseplate', 'kegg_head', 'kegg_tail', 'kegg_tail_fiber', 'kegg_terminase', 'kegg_integrase', 'kegg_transposase', 'viral_capsid', 'viral_portal', 'viral_coat', 'viral_baseplate', 'viral_head', 'viral_tail', 'viral_tail_fiber', 'viral_terminase', 'viral_integrase', 'viral_transposase', 'pfam_capsid', 'pfam_portal', 'pfam_coat', 'pfam_baseplate', 'pfam_head', 'pfam_tail', 'pfam_tail_fiber', 'pfam_terminase', 'pfam_integrase', 'pfam_transposase'])


In [22]:
dramv_amg_for_manual_curation_final.reset_index(inplace = True)
dramv_amg_for_manual_curation_final.rename(columns = {'index': 'contig'}, inplace = True)

In [23]:
amg_manual_curation = pd.merge(dramv_amg_for_manual_curation_final, checkv_for_amg_curation, left_on = 'contig', right_on = 'contig_id').drop('contig_id', axis = 1)

In [24]:
def remove_bracket(x):
    if str(x).startswith(r'['):
        return re.sub(r"[\[\]']", '', x)
    else:
        return x
    
            
amg_manual_curation = amg_manual_curation.applymap(remove_bracket)

In [25]:
amg_manual_curation['all_structure'] = amg_manual_curation['kegg_capsid'] + amg_manual_curation['kegg_portal'] + amg_manual_curation['kegg_coat'] + amg_manual_curation['kegg_baseplate'] + amg_manual_curation['kegg_head'] + amg_manual_curation['kegg_tail'] + amg_manual_curation['kegg_tail_fiber'] + amg_manual_curation['kegg_terminase'] +  amg_manual_curation['viral_capsid'] + amg_manual_curation['viral_portal'] + amg_manual_curation['viral_coat'] + amg_manual_curation['viral_baseplate'] + amg_manual_curation['viral_head'] + amg_manual_curation['viral_tail'] + amg_manual_curation['viral_tail_fiber'] + amg_manual_curation['viral_terminase'] + amg_manual_curation['pfam_capsid'] + amg_manual_curation['pfam_portal'] + amg_manual_curation['pfam_coat'] + amg_manual_curation['pfam_baseplate'] + amg_manual_curation['pfam_head'] + amg_manual_curation['pfam_tail'] + amg_manual_curation['pfam_tail_fiber'] + amg_manual_curation['pfam_terminase'] 

In [26]:
amg_manual_curation['structure_no_tail'] = amg_manual_curation['kegg_capsid'] + amg_manual_curation['kegg_portal'] + amg_manual_curation['kegg_coat'] + amg_manual_curation['kegg_baseplate'] + amg_manual_curation['kegg_head'] + amg_manual_curation['kegg_terminase'] +  amg_manual_curation['viral_capsid'] + amg_manual_curation['viral_portal'] + amg_manual_curation['viral_coat'] + amg_manual_curation['viral_baseplate'] + amg_manual_curation['viral_head'] + amg_manual_curation['viral_terminase'] + amg_manual_curation['pfam_capsid'] + amg_manual_curation['pfam_portal'] + amg_manual_curation['pfam_coat'] + amg_manual_curation['pfam_baseplate'] + amg_manual_curation['pfam_head']  + amg_manual_curation['pfam_terminase'] 

In [27]:
amg_manual_curation['integrase_or_transposase'] = amg_manual_curation['kegg_integrase'] + amg_manual_curation['kegg_transposase'] + amg_manual_curation['viral_integrase'] + amg_manual_curation['viral_transposase'] + amg_manual_curation['pfam_integrase'] + amg_manual_curation['pfam_transposase'] 

In [28]:
#amg_manual_curation.to_csv('/fs/ess/PAS0439/MING/virome/AMR_AMG_containing_contigs_manual_check/amg_manual_curation.csv', index = None)

In [56]:
os.chdir('/fs/scratch/PAS0439/Ming/results/amr_results/dramv')

dramv_amr = glob.glob('**/annotations.tsv', recursive = True)
dramv_amr_df = []

for file in dramv_amr:
    df = pd.read_csv(file, sep = '\t')
    dramv_amr_df.append(df)
    dramv_amr_for_manual_curation = pd.concat(dramv_amr_df)

dramv_amr_for_manual_curation.rename(columns = {'Unnamed: 0': 'gene_name'}, inplace = True)
dramv_amr_for_manual_curation.fasta = dramv_amr_for_manual_curation.fasta.apply(lambda x: x.replace('__full-cat_1', '||full'))
dramv_amr_for_manual_curation.fasta = dramv_amr_for_manual_curation.fasta.apply(lambda x: x.replace('__full-cat_2', '||full'))

dramv_amr_for_manual_curation_reformat = dramv_amr_for_manual_curation.get(['gene_name', 'fasta', 'kegg_hit', 'viral_hit', 'pfam_hits'])
dramv_amr_for_manual_curation_reformat_new = dramv_amr_for_manual_curation_reformat[dramv_amr_for_manual_curation_reformat.fasta.isin(amr_contigs_for_checking)]

group = dramv_amr_for_manual_curation_reformat_new.groupby('fasta').groups.keys()

group = dramv_amr_for_manual_curation_reformat_new.groupby('fasta').groups.keys()
dramv_amr_for_manual_curation_final = pd.DataFrame(list(dramv_amr_for_manual_curation_reformat_new.groupby('fasta').apply(get_hallmark)), index = group, columns = ['kegg_capsid', 'kegg_portal', 'kegg_coat', 'kegg_baseplate', 'kegg_head', 'kegg_tail', 'kegg_tail_fiber', 'kegg_terminase', 'kegg_integrase', 'kegg_transposase', 'viral_capsid', 'viral_portal', 'viral_coat', 'viral_baseplate', 'viral_head', 'viral_tail', 'viral_tail_fiber', 'viral_terminase', 'viral_integrase', 'viral_transposase', 'pfam_capsid', 'pfam_portal', 'pfam_coat', 'pfam_baseplate', 'pfam_head', 'pfam_tail', 'pfam_tail_fiber', 'pfam_terminase', 'pfam_integrase', 'pfam_transposase'])

dramv_amr_for_manual_curation_final.reset_index(inplace = True)
dramv_amr_for_manual_curation_final.rename(columns = {'index': 'contig'}, inplace = True)

amr_manual_curation = pd.merge(dramv_amr_for_manual_curation_final, checkv_for_amr_curation, left_on = 'contig', right_on = 'contig_id').drop('contig_id', axis = 1)

amr_manual_curation = amr_manual_curation.applymap(remove_bracket)

amr_manual_curation['all_structure'] = amr_manual_curation['kegg_capsid'] + amr_manual_curation['kegg_portal'] + amr_manual_curation['kegg_coat'] + amr_manual_curation['kegg_baseplate'] + amr_manual_curation['kegg_head'] + amr_manual_curation['kegg_tail'] + amr_manual_curation['kegg_tail_fiber'] + amr_manual_curation['kegg_terminase'] +  amr_manual_curation['viral_capsid'] + amr_manual_curation['viral_portal'] + amr_manual_curation['viral_coat'] + amr_manual_curation['viral_baseplate'] + amr_manual_curation['viral_head'] + amr_manual_curation['viral_tail'] + amr_manual_curation['viral_tail_fiber'] + amr_manual_curation['viral_terminase'] + amr_manual_curation['pfam_capsid'] + amr_manual_curation['pfam_portal'] + amr_manual_curation['pfam_coat'] + amr_manual_curation['pfam_baseplate'] + amr_manual_curation['pfam_head'] + amr_manual_curation['pfam_tail'] + amr_manual_curation['pfam_tail_fiber'] + amr_manual_curation['pfam_terminase'] 
amr_manual_curation['structure_no_tail'] = amr_manual_curation['kegg_capsid'] + amr_manual_curation['kegg_portal'] + amr_manual_curation['kegg_coat'] + amr_manual_curation['kegg_baseplate'] + amr_manual_curation['kegg_head'] + amr_manual_curation['kegg_terminase'] +  amr_manual_curation['viral_capsid'] + amr_manual_curation['viral_portal'] + amr_manual_curation['viral_coat'] + amr_manual_curation['viral_baseplate'] + amr_manual_curation['viral_head'] + amr_manual_curation['viral_terminase'] + amr_manual_curation['pfam_capsid'] + amr_manual_curation['pfam_portal'] + amr_manual_curation['pfam_coat'] + amr_manual_curation['pfam_baseplate'] + amr_manual_curation['pfam_head']  + amr_manual_curation['pfam_terminase'] 
amr_manual_curation['integrase_or_transposase'] = amr_manual_curation['kegg_integrase'] + amr_manual_curation['kegg_transposase'] + amr_manual_curation['viral_integrase'] + amr_manual_curation['viral_transposase'] + amr_manual_curation['pfam_integrase'] + amr_manual_curation['pfam_transposase'] 

In [59]:
amr_manual_curation.query('contig_length > 10000', inplace = True)

In [61]:
amr_manual_curation.to_csv('/fs/ess/PAS0439/MING/virome/AMR_AMG_containing_contigs_manual_check/amr_manual_curation.csv', index = None)